In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import scipy.stats as stats

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from acquire import get_zillow_data

In [2]:
df = get_zillow_data()

/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (64) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,6.059063e+13,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,6.111001e+13,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,6.059022e+13,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,6.037300e+13,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,6.037124e+13,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [4]:
df = df[df.groupby('parcelid')['transactiondate'].transform('max') == df['transactiondate']]


In [5]:
df.shape

(77381, 68)

In [6]:
new_df = df[df['unitcnt'] == 1]

In [7]:
new_df.shape

(47293, 68)

In [8]:
def handle_missing_values(df, prop_required_column = .60, prop_required_row = .75):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

In [9]:
new_df = handle_missing_values(new_df)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [10]:
new_df.shape

(47293, 34)

In [11]:
def remove_columns(df, cols_to_remove):  
    df = df.drop(columns=cols_to_remove)
    return df

**Going to convert fips to county name and year built to age**

In [12]:
new_df.loc[(new_df.fips == 6037.0),'fips']='Los Angeles County'

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [13]:
new_df.loc[(new_df.fips == 6111.0),'fips']='Ventura County'

In [14]:
new_df.loc[(new_df.fips == 6059.0),'fips']='Orange County'

In [15]:
new_df['age'] = 2020 - new_df['yearbuilt']

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
new_df.head()

,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc,age
3,2288172,12177905,3.0,4.0,8.0,3.0,2376.0,2376.0,Los Angeles County,3.0,...,145143.0,2016.0,36225.0,1777.51,6.037300e+13,-0.103410,2017-01-01,Central,Single Family Residential,50.0
4,1970746,10887214,3.0,3.0,8.0,3.0,1312.0,1312.0,Los Angeles County,3.0,...,119407.0,2016.0,45726.0,1533.89,6.037124e+13,0.006940,2017-01-01,Central,Condominium,56.0
6,781532,12095076,3.0,4.0,9.0,3.0,2962.0,2962.0,Los Angeles County,3.0,...,773303.0,2016.0,496619.0,9516.26,6.037461e+13,-0.001011,2017-01-01,Central,Single Family Residential,70.0
7,870991,12069064,1.0,2.0,5.0,1.0,738.0,738.0,Los Angeles County,1.0,...,218552.0,2016.0,199662.0,2366.08,6.037302e+13,0.101723,2017-01-01,NaN,Single Family Residential,98.0
8,1246926,12790562,3.0,4.0,9.0,3.0,3039.0,3039.0,Los Angeles County,3.0,...,220583.0,2016.0,43056.0,3104.19,6.037500e+13,-0.040966,2017-01-02,Central,Single Family Residential,50.0


Going to consolidate the taxvaluedollarcount and taxamount by creating a tax percentage column.

In [17]:
new_df['tax_percentage'] = round(new_df['taxamount'] / new_df['taxvaluedollarcnt'],4)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
new_df.head()

,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc,age,tax_percentage
3,2288172,12177905,3.0,4.0,8.0,3.0,2376.0,2376.0,Los Angeles County,3.0,...,2016.0,36225.0,1777.51,6.037300e+13,-0.103410,2017-01-01,Central,Single Family Residential,50.0,0.0122
4,1970746,10887214,3.0,3.0,8.0,3.0,1312.0,1312.0,Los Angeles County,3.0,...,2016.0,45726.0,1533.89,6.037124e+13,0.006940,2017-01-01,Central,Condominium,56.0,0.0128
6,781532,12095076,3.0,4.0,9.0,3.0,2962.0,2962.0,Los Angeles County,3.0,...,2016.0,496619.0,9516.26,6.037461e+13,-0.001011,2017-01-01,Central,Single Family Residential,70.0,0.0123
7,870991,12069064,1.0,2.0,5.0,1.0,738.0,738.0,Los Angeles County,1.0,...,2016.0,199662.0,2366.08,6.037302e+13,0.101723,2017-01-01,NaN,Single Family Residential,98.0,0.0108
8,1246926,12790562,3.0,4.0,9.0,3.0,3039.0,3039.0,Los Angeles County,3.0,...,2016.0,43056.0,3104.19,6.037500e+13,-0.040966,2017-01-02,Central,Single Family Residential,50.0,0.0141


In [19]:
new_df = remove_columns(new_df, ['id','parcelid','finishedsquarefeet12', 'regionidzip', 'censustractandblock','calculatedbathnbr','fullbathcnt', 'regionidcity' , 'structuretaxvaluedollarcnt', 'landtaxvaluedollarcnt','propertycountylandusecode', 'propertylandusetypeid', 'regionidcounty' , 'assessmentyear', 'propertyzoningdesc','transactiondate','heatingorsystemtypeid', 'buildingqualitytypeid','yearbuilt', 'taxamount', 'taxvaluedollarcnt' ])

**Going to split data before we impute values because imputed values will be calculated on training data**

# Train, Test, Split

In [20]:
train_validate, test = train_test_split(new_df, test_size=.2, 
                                        random_state=42)
train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=42)

print("train observations: ", train.size)
print("validate observations: ", validate.size)
print("test observations: ", test.size)

train observations:  397245
validate observations:  170265
test observations:  141885


In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26483 entries, 14520 to 29445
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   26483 non-null  float64
 1   bedroomcnt                    26483 non-null  float64
 2   calculatedfinishedsquarefeet  26478 non-null  float64
 3   fips                          26483 non-null  object 
 4   latitude                      26483 non-null  float64
 5   longitude                     26483 non-null  float64
 6   lotsizesquarefeet             25765 non-null  float64
 7   rawcensustractandblock        26483 non-null  float64
 8   roomcnt                       26483 non-null  float64
 9   unitcnt                       26483 non-null  float64
 10  logerror                      26483 non-null  float64
 11  heatingorsystemdesc           26072 non-null  object 
 12  propertylandusedesc           26483 non-null  object 
 1

**Search for columns with missing values, going to impute mean based on train**

**Columns w/ missing values**
- calculatedfinishedsquarefeet
- lotsizesquarefeet

- heatingorsystemdesc
- age
- tax_percentage

In [22]:
train.calculatedfinishedsquarefeet.mean()

1700.3954603822042

In [23]:
train.lotsizesquarefeet.mean()

39946.982844944694

In [24]:
train.heatingorsystemdesc.value_counts()

Central       18572
Floor/Wall     7438
Solar            62
Name: heatingorsystemdesc, dtype: int64